In [1]:
#Credit yuanyuanshi
#https://github.com/yuanyuanshi/Data_Skills/blob/master/data_skills_2.py

#Credit:  Massive props to Byron Stuart for helping me getting this code running

import re
from nltk.corpus import stopwords #conda install nltk

#Import Goose here: https://pypi.python.org/pypi/goose-extractor/
#Follow instructions.  It is NOT a straight forward pip install
from goose import Goose 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
import time
from scrapy.selector import Selector
import requests
import random
import pandas as pd
import matplotlib.pyplot as plt

#This will only work in jupyter notebook and command line. I tried in ipython and it did not work.

In [2]:
#I get these keywords from the first page search result of data scientist at indeed; they're not whole but already tell a story.
program_languages=['bash','r','python','java','c++','ruby','perl','matlab','javascript','scala','php']
analysis_software=['excel','tableau','d3.js','sas','spss','d3','saas','pandas','numpy','scipy','sps','spotfire','scikits.learn','splunk','powerpoint','h2o']
bigdata_tool=['hadoop','mapreduce','spark','pig','hive','shark','oozie','zookeeper','flume','mahout']
databases=['sql','nosql','hbase','cassandra','mongodb','mysql','mssql','postgresql','oracle db','rdbms']
education = ['bachelors','bachelor','masters','phd', 'bsc','msc']
skills = ['machine learning', 'svm', 'source vector modeling', 'random forest', 'neural network', 'decision trees','mining', 'mining', 'web scraping']
overall_dict = program_languages + analysis_software + bigdata_tool + databases + education + skills


#extra skills to add = ai, artificial intelligence, tensorflow, nlp, natural language processing,
#machine learning, svm, source vector modeling, random forest, neural network, ensemble, decision trees
#data mining, mining, webscraping, web scraping, communication skills

In [3]:
# the following two functions are for webpage text processing to extract the skill keywords.
def keywords_extract(url):
    g = Goose()
    article = g.extract(url=url)
    text = article.cleaned_text
    text = re.sub("[^a-zA-Z+3]"," ", text) #get rid of things that aren't words; 3 for d3 and + for c++
    text = text.lower().split()
    stops = set(stopwords.words("english")) #filter out stop words in english language
    text = [w for w in text if not w in stops]
    text = list(set(text))
    keywords = [str(word) for word in text if word in overall_dict]
    return keywords

In [4]:
#for this function, thanks to this blog:https://jessesw.com/Data-Science-Skills/
def keywords_f(soup_obj):
    for script in soup_obj(["script", "style"]):
        script.extract() # Remove these two elements from the BS4 object
    text = soup_obj.get_text()
    lines = (line.strip() for line in text.splitlines()) # break into line
    chunks = (phrase.strip() for line in lines for phrase in line.split("  ")) # break multi-headlines into a line each
    text = ''.join(chunk for chunk in chunks if chunk).encode('utf-8') # Get rid of all blank lines and ends of line
    try:
        text = text.decode('unicode_escape').encode('ascii', 'ignore') # Need this as some websites aren't formatted
    except:
        return
    text = re.sub("[^a-zA-Z+3]"," ", text)
    text = re.sub(r"([a-z])([A-Z])", r"\1 \2", text) # Fix spacing issue from merged words
    text = text.lower().split()  # Go to lower case and split them apart
    stop_words = set(stopwords.words("english")) # Filter out any stop words
    text = [w for w in text if not w in stop_words]
    text = list(set(text)) #only care about if a word appears, don't care about the frequency
    keywords = [str(word) for word in text if word in overall_dict] #if a skill keyword is found, return it.
    return keywords


In [5]:
base_url = "https://www.seek.com.au"

#change the start_url can scrape different cities.
#start_url = 'https://www.seek.com.au/Data-Scientist-jobs?page=7'
start_url = 'https://www.seek.com.au/Data-Analyst-jobs?page=1'

resp = requests.get(start_url)
start_soup = BeautifulSoup(resp.content,'html.parser')

domain = 'https://www.seek.com.au'

#Find number urls (jobs on first page)
urls=[]
for entry in start_soup.find_all('a',{'class':'_1OFaluu'},href=True):
    urls.append(domain+entry['href'])

#When we search for data science jobs we see there are 7 pages of jobs
#Will eventuallyuse a function to cycle through each page


#Create list of the pages
job_page=[]
for u in range(1,8):
    job_page.append(start_url+'?'+'page='+str(u))
job_page

['https://www.seek.com.au/Data-Analyst-jobs?page=1?page=1',
 'https://www.seek.com.au/Data-Analyst-jobs?page=1?page=2',
 'https://www.seek.com.au/Data-Analyst-jobs?page=1?page=3',
 'https://www.seek.com.au/Data-Analyst-jobs?page=1?page=4',
 'https://www.seek.com.au/Data-Analyst-jobs?page=1?page=5',
 'https://www.seek.com.au/Data-Analyst-jobs?page=1?page=6',
 'https://www.seek.com.au/Data-Analyst-jobs?page=1?page=7']

In [22]:
#From Eric Nessi

def job_func(job_title='Data+Analyst', city = None, state = None, page_number=None):
# job_title ='Data+Analyst' 
# city = None
# state = None
# page_number=None
    
    # Make sure the city specified works properly if it has more than one word (such as San Francisco)
    if city is not None:
        final_city = city.split()
        final_city = '+'.join(word for word in final_city)
        # Join all of our strings together so that indeed will search correctly        start_url_list = ['https://www.seek.com.au/jobs/in-All-', final_city, '-', state, '?keywords=%22', job_title, '%22']
    else:
        start_url_list = ['https://www.seek.com.au/jobs?keywords=%22', job_title, '%22']
    
    start_url = ''.join(start_url_list)    
    #change the start_url can scrape different cities.

    domain = 'https://www.seek.com.au'

    #Find number urls (jobs on first page)
    page_number = 20
    #page_number + 1
    
    job_page = []
    job_page.append(start_url)
    for u in range(2,page_number):
        job_page.append(start_url+'&'+'page='+str(u))

In [23]:
#data_analyst_urls = 
job_func(job_title='Data+Analyst', city = None, state = None, page_number = 20)
#len(data_analyst_urls)
#data_analyst_urls[0:5]

In [80]:
data_sci_urls = job_func(job_title='Data+Scientist', city = None, state = None, page_number = 8)
len(data_sci_urls)

TypeError: object of type 'NoneType' has no len()

In [47]:
analytics_con_urls = job_func(job_title='Analytics+Consultant', city = None, state = None, page_number = 5)
len(analytics_con_urls)

36

In [51]:
bus_intel_urls = job_func(job_title='business+intelligence', city = None, state = None, page_number = 38)
len(bus_intel_urls)

704

In [8]:
#Merge all urls
urls_merge = data_analyst_urls + data_sci_urls + analytics_con_urls + bus_intel_urls
len(urls_merge)

#Looks like we have over 1000 jobs

NameError: name 'data_sci_urls' is not defined

In [68]:
#Check for duplicate urls
urls_df = pd.DataFrame({'urls':urls_merge})
print urls_df.shape
urls_df.head()

(1124, 1)


,urls
0,https://www.seek.com.au/job/33431855?type=stan...
1,https://www.seek.com.au/job/33431139?type=stan...
2,https://www.seek.com.au/job/33431716?type=stan...
3,https://www.seek.com.au/job/33430694?type=stan...
4,https://www.seek.com.au/job/33428799?type=stan...


In [199]:
(urls_df).drop_duplicates()
urls_df.shape

#no duplicate urls

(1124, 1)

In [200]:
urls=urls_merge

In [201]:
urls=urls_merge
len(urls)

1124

In [115]:
urls_df.to_csv('./urls_df.csv')
#a.to_csv("test.csv"

In [16]:
'''
base_url = "https://www.seek.com.au"

#change the start_url can scrape different cities.
#start_url = 'https://www.seek.com.au/Data-Scientist-jobs?page=7'
start_url = 'https://www.seek.com.au/Data-Analyst-jobs?page=1'

resp = requests.get(start_url)
start_soup = BeautifulSoup(resp.content,'html.parser')

domain = 'https://www.seek.com.au'

#Find number urls (jobs on first page)
urls=[]
for entry in start_soup.find_all('a',{'class':'_1OFaluu'},href=True):
    urls.append(domain+entry['href'])

#When we search for data science jobs we see there are 7 pages of jobs
#Will eventuallyuse a function to cycle through each page


#Create list of the pages
job_page=[]
for u in range(1,8):
    job_page.append(start_url+'?'+'page='+str(u))
job_page
'''

['https://www.seek.com.au/Data-Analyst-jobs?page=1?page=1',
 'https://www.seek.com.au/Data-Analyst-jobs?page=1?page=2',
 'https://www.seek.com.au/Data-Analyst-jobs?page=1?page=3',
 'https://www.seek.com.au/Data-Analyst-jobs?page=1?page=4',
 'https://www.seek.com.au/Data-Analyst-jobs?page=1?page=5',
 'https://www.seek.com.au/Data-Analyst-jobs?page=1?page=6',
 'https://www.seek.com.au/Data-Analyst-jobs?page=1?page=7']

In [96]:
urls_csv = '/Users/gopetro/documents/dsi/materials/project4/urls1.csv'
import csv
with open(urls_csv, 'r') as f:
    reader = csv.reader(f)
    urls = list(reader)

#Flatten list
from itertools import chain

urls = list(chain.from_iterable(urls))
    
#urls=urls[1:]
urls[0:3]

['urls',
 'https://www.seek.com.au/job/33431855?type=standout&tier=no_tier&pos=1&whereid=3000&userqueryid=e8af3ba9d327b21d8e0161b035a84567-7472012&ref=beta',
 'https://www.seek.com.au/job/33431139?type=standout&tier=no_tier&pos=2&whereid=3000&userqueryid=e8af3ba9d327b21d8e0161b035a84567-7472012&ref=beta']

In [196]:
# open Chrome driver
driver = webdriver.Chrome(executable_path="/Users/gopetro/documents/dsi/code/chromedriver")
get_info = True

#Firefox
#webdriver.DesiredCapabilities.FIREFOX["unexpectedAlertBehaviour"] = "accept"
#get_info = True
#driver=webdriver.Firefox(executable_path="/Users/gopetro/documents/dsi/code/geckodriver")

# set a page load time limit so that don't have to wait forever if the links are broken.
driver.set_page_load_timeout(15)

In [206]:
#Split urls
#urls1 = urls[0:140]
# urls2 = urls[162:280]
# urls3 = urls[281:420]
# urls3a = urls[394:420]
# urls4 = urls[421:560]
# urls4a = urls[445:560]
# urls5 = urls[561:700]
# urls5a= urls[631:700]
# urls5b= urls[667:700]
# urls6 = urls[701:840]
# urls7 = urls[841:980]
# urls7a = urls[909:981]
# urls8 = urls[981:1124]
# urls8a=urls[1007:1124]

#urls2a = urls[258:280]
urls3b = urls[345:420]
urls3c = urls[406:420]
urls4a = urls[490:560]
urls4b = urls[513:560]
urls4c = urls[522:560]
urls5a= urls[650:700]
#urls6 = urls[581:700]
urls6a = urls[750:850]

#urls5b= urls[667:700]
#urls6 = urls[701:840]
#urls7 = urls[841:980]
#urls7a = urls[909:981]
#urls8 = urls[981:1124]
#urls8a=urls[1007:1124]

In [207]:
#As we run search let's create list for each.  We will merge each list at the end to make dataframe
Date=[]
Title=[]
Location=[]
Salary =[]
Employment_Type =[]
Industry =[]
skills = []
Title =[]
Education =[]
job_keywords=[]

len(urls6a)

100

In [208]:
#Base code for doing a scan of one page
#THis one works

#for i in range(1,4):
#for i in range(5,len(urls))
#for i in range(len(urls)):
for i in range(1,len(urls6a)):    
    get_info = True
    try:
        #requests.get(urls[i].text)
        driver.get(urls6a[i])
    except TimeoutException:
        get_info = False
        continue
    j = random.randint(1000,5000)/1000.0
    time.sleep(j) #waits for a random time so that the website don't consider you as a bot
    if get_info:
        soup=BeautifulSoup(driver.page_source, 'html.parser')
        print 'extracting %d job keywords...' % i
        single_job = keywords_f(soup)
        print single_job,len(soup)
        #print driver.current_url
        #job_keywords.append([driver.urls[i],single_job])
        skills.append(single_job)
        
        #job_keywords.append([driver.current_url,single_job])
        #skills.append(single_job)
        
        #Byron's addition
        #print 'Date posted'
        #print soup.find('strong', {'itemprop': 'datePosted'}).get_text(" ", strip=True)
        Date.append(soup.find('strong', {'itemprop': 'datePosted'}).get_text(" ", strip=True))
        #print 'Job title:', soup.find('h1').get_text(" ", strip=True)
        Title.append(soup.find('h1').get_text(" ", strip=True))
            #print(job_page.find('h1', {'class': 'jobtitle'}).get_text(" ", strip=True))
        
        #print soup.find('li', {'itemprop': 'jobLocation'}).get_text(" ", strip=True)
        Location.append(soup.find('li', {'itemprop': 'jobLocation'}).get_text(" ", strip=True))

        if soup.find('div', {'itemprop': 'baseSalary'}) == None:
        #    print 'Salary:', 'None stated'
            Salary.append(soup.find('div', {'itemprop': 'baseSalary'}))
        else:
        #    print 'Salary:', soup.find('div', {'itemprop': 'baseSalary'}).get_text(" ", strip=True)
            Salary.append(soup.find('div', {'itemprop': 'baseSalary'}).get_text(" ", strip=True))
        
        
        #print 'Employment Type:', soup.find('div', {'itemprop': 'employmentType'}).get_text(" ", strip=True)
        Employment_Type.append(soup.find('div', {'itemprop': 'employmentType'}).get_text(" ", strip=True))
            
        
        
        #print 'Industry:', soup.find('span', {'itemprop': 'industry'}).get_text(" ", strip=True)
        Industry.append(soup.find('span', {'itemprop': 'industry'}).get_text(" ", strip=True))
		#	mod_job_details = job_page.find('div', {'class': 'mod-job-details'})
        #    mod_job_details = mod_job_details.find_all('div')
        #    div_list = list(mod_job_details)
        #print 'Industry:', div_list[len(div_list)-1].get_text(" ", strip=True)
        df6a=pd.DataFrame({'Date_Posted':Date,
                             'Location':Location,
                             'Title':Title,
                             'Type':Employment_Type,
                             'Salary':Salary,
                             'Industry':Industry,
                             'Skills':skills})
        
        #df.head()
        df6a.to_csv('./urls_df6a.csv')

extracting 1 job keywords...
['excel'] 4
extracting 2 job keywords...
[] 4
extracting 3 job keywords...
['tableau', 'python', 'r', 'sas', 'spark', 'hadoop', 'scala', 'spss'] 4
extracting 4 job keywords...
['tableau', 'python', 'sas', 'java', 'splunk', 'sql'] 4
extracting 5 job keywords...
['hadoop', 'sql'] 4
extracting 6 job keywords...
[] 4
extracting 7 job keywords...
['tableau', 'sql'] 4
extracting 8 job keywords...
['java', 'sql'] 4
extracting 9 job keywords...
['excel', 'tableau', 'sql'] 4
extracting 10 job keywords...
['excel'] 4
extracting 11 job keywords...
['tableau', 'r', 'sas'] 4


UnicodeEncodeError: 'ascii' codec can't encode character u'\u2013' in position 35: ordinal not in range(128)

In [206]:
df2.shape
df2.to_csv('./urls_df2.csv')

In [80]:
urls[1]

['https://www.seek.com.au/job/33431139?type=standout&tier=no_tier&pos=2&whereid=3000&userqueryid=e8af3ba9d327b21d8e0161b035a84567-7472012&ref=beta']

In [79]:
#Base code for doing a scan of one page

#for i in range(1,4):
#for i in range(5,len(urls))
#for i in range(len(urls)):
for i in range(1,len(urls4a)):    
    get_info = True
    try:
        requests.get(urls[i].text)
        #driver.get(urls[i])
    except TimeoutException:
        get_info = False
        continue
    j = random.randint(1000,2200)/1000.0
    time.sleep(j) #waits for a random time so that the website don't consider you as a bot
    if get_info:
        soup=BeautifulSoup(driver.page_source, 'html.parser')
        print 'extracting %d job keywords...' % i
        single_job = keywords_f(soup)
        print single_job,len(soup)
        #print driver.current_url
        #job_keywords.append([driver.urls[i],single_job])
        skills.append(single_job)
        
        #job_keywords.append([driver.current_url,single_job])
        #skills.append(single_job)
        
        #Byron's addition
        #print 'Date posted'
        #print soup.find('strong', {'itemprop': 'datePosted'}).get_text(" ", strip=True)
        Date.append(soup.find('strong', {'itemprop': 'datePosted'}).get_text(" ", strip=True))
        #print 'Job title:', soup.find('h1').get_text(" ", strip=True)
        Title.append(soup.find('h1').get_text(" ", strip=True))
            #print(job_page.find('h1', {'class': 'jobtitle'}).get_text(" ", strip=True))
        
        #print soup.find('li', {'itemprop': 'jobLocation'}).get_text(" ", strip=True)
        Location.append(soup.find('li', {'itemprop': 'jobLocation'}).get_text(" ", strip=True))

        if soup.find('div', {'itemprop': 'baseSalary'}) == None:
        #    print 'Salary:', 'None stated'
            Salary.append(soup.find('div', {'itemprop': 'baseSalary'}))
        else:
        #    print 'Salary:', soup.find('div', {'itemprop': 'baseSalary'}).get_text(" ", strip=True)
            Salary.append(soup.find('div', {'itemprop': 'baseSalary'}).get_text(" ", strip=True))
        
        
        #print 'Employment Type:', soup.find('div', {'itemprop': 'employmentType'}).get_text(" ", strip=True)
        Employment_Type.append(soup.find('div', {'itemprop': 'employmentType'}).get_text(" ", strip=True))
            
        
        
        #print 'Industry:', soup.find('span', {'itemprop': 'industry'}).get_text(" ", strip=True)
        Industry.append(soup.find('span', {'itemprop': 'industry'}).get_text(" ", strip=True))
		#	mod_job_details = job_page.find('div', {'class': 'mod-job-details'})
        #    mod_job_details = mod_job_details.find_all('div')
        #    div_list = list(mod_job_details)
        #print 'Industry:', div_list[len(div_list)-1].get_text(" ", strip=True)
        df4a=pd.DataFrame({'Date_Posted':Date,
                             'Location':Location,
                             'Title':Title,
                             'Type':Employment_Type,
                             'Salary':Salary,
                             'Industry':Industry,
                             'Skills':skills})
        
        #df.head()
        df4a.to_csv('./urls_df.csv')

AttributeError: 'list' object has no attribute 'text'

In [165]:
len(urls)

20

In [264]:
#yens code
## initialize DataFrame
data = pd.DataFrame(columns=['title','salary','location','advertiser', 'industry','description'])

def get_job_info(job_url):
    html = requests.get(job_url)
    soup = BeautifulSoup(html.content, 'html.parser')
    # Get the row
    try:
        title = soup.find("h1","jobtitle").string
    except Exception:
        title = 'NaN'
    # to take into account cases where salary is not listed, I want to write
    #salary = NaN if soup.find("div", {'itemprop':"baseSalary"}).renderContents() not valid
    try:
        salary = soup.find("div", {'itemprop':"baseSalary"}).renderContents()
    except Exception:
        salary = 'NaN'
    
    try:
        location = soup.find("span", {'data-automation':'apply_text_location_area'}).renderContents()
    except Exception:
        location = 'NaN'
    try:
        advertiser = soup.find("span", {'class':'state-message'}).renderContents()
    except Exception:
        advertiser = 'NaN'
    try:
        industry = soup.find("span", {'itemprop':'industry'}).renderContents()
    except Exception:
        industry = 'NaN'
    try:
        description = soup.find("div", {'class':'templatetext'}).renderContents()
    except Exception:
        description = 'NaN'   
    # add to data dataframe
    data.loc[len(data.title)]=[title,salary,location,advertiser,industry,description]
    # add a counter so I can see what's happenning
    if len(data.title) >0:
        print(len(data.title))
    data.to_csv('./data_df3.csv')
    
#        single_job = keywords_f(soup)
#        print single_job,len(soup)
#        #print driver.current_url
#        #job_keywords.append([driver.urls[i],single_job])
#        skills.append(single_job)    

In [265]:
len(urls3)


139

In [262]:
for i in range(10,len(urls3)):
    get_job_info(urls3[i])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [127]:
len(df)
df.shape

(161, 7)

In [310]:
'''
frames = [df1, df2, df3, df4, df5, df6, df7]

ds_result = pd.concat(frames)
ds_result.to_csv('ds_result.csv')
print ds_result.shape
ds_result.head(30)
'''

"\nframes = [df1, df2, df3, df4, df5, df6, df7]\n\nds_result = pd.concat(frames)\nds_result.to_csv('ds_result.csv')\nprint ds_result.shape\nds_result.head(30)\n"

In [10]:
#This function gets the job listing page, gets the url link for each job and scans the information
def page(job_page):
    #job_page is first page to scan
    
    start_url = job_page #Set the page with jobs
    resp = requests.get(start_url)
    start_soup = BeautifulSoup(resp.content,'html.parser') #Use beautifulsoup to scrape the page

    domain = 'https://www.seek.com.au'

    #Find number urls (jobs on first page)
    urls=[]
    for entry in start_soup.find_all('a',{'class':'_1OFaluu'},href=True):
        urls.append(domain+entry['href'])
    print len(urls)
  
    # open the driver
    driver = webdriver.Chrome(executable_path="/Users/gopetro/documents/dsi/code/chromedriver")
    get_info = True

    # set a page load time limit so that don't have to wait forever if the links are broken.
    driver.set_page_load_timeout(15)
    
    #As we run search need to reset each list to maintain same dimensions
    #We will merge each list at the end to make dataframe
    Date=[]
    Title=[]
    Location=[]
    Salary =[]
    Employment_Type =[]
    Industry =[]
    skills = []
    Title =[]
    Education =[]
    job_keywords=[]
    
    for i in range(0,2):
    #for i in range(len(urls)):
        get_info = True
        try:
            driver.get(urls[i])
        except TimeoutException:
            get_info = False
            continue
        j = random.randint(1000,2200)/1000.0
        time.sleep(j) #waits for a random time so that the website don't consider you as a bot
        
        if get_info:
            
            soup=BeautifulSoup(driver.page_source, 'html.parser')
            print 'extracting %d job keywords...' % i
            single_job = keywords_f(soup)
            print single_job,len(soup)
            #print driver.current_url
            #job_keywords.append([driver.urls[i],single_job])
            skills.append(single_job)
            
            #Byron's addition
            #print 'Date posted'
            #print soup.find('strong', {'itemprop': 'datePosted'}).get_text(" ", strip=True)
            Date.append(soup.find('strong', {'itemprop': 'datePosted'}).get_text(" ", strip=True))
            #print 'Job title:', soup.find('h1').get_text(" ", strip=True)
            Title.append(soup.find('h1').get_text(" ", strip=True))
            #print(job_page.find('h1', {'class': 'jobtitle'}).get_text(" ", strip=True))
        
            #print soup.find('li', {'itemprop': 'jobLocation'}).get_text(" ", strip=True)
            Location.append(soup.find('li', {'itemprop': 'jobLocation'}).get_text(" ", strip=True))

            if soup.find('div', {'itemprop': 'baseSalary'}) == None:
                #print 'Salary:', 'None stated'
                Salary.append(soup.find('div', {'itemprop': 'baseSalary'}))
            else:
                #print 'Salary:', soup.find('div', {'itemprop': 'baseSalary'}).get_text(" ", strip=True)
                Salary.append(soup.find('div', {'itemprop': 'baseSalary'}).get_text(" ", strip=True))
        
        
            #print 'Employment Type:', soup.find('div', {'itemprop': 'employmentType'}).get_text(" ", strip=True)
            Employment_Type.append(soup.find('div', {'itemprop': 'employmentType'}).get_text(" ", strip=True))
            
        
        
            #print 'Industry:', soup.find('span', {'itemprop': 'industry'}).get_text(" ", strip=True)
            Industry.append(soup.find('span', {'itemprop': 'industry'}).get_text(" ", strip=True))
            #mod_job_details = job_page.find('div', {'class': 'mod-job-details'})
            #mod_job_details = mod_job_details.find_all('div')
            #div_list = list(mod_job_details)
            #print 'Industry:', div_list[len(div_list)-1].get_text(" ", strip=True)
#     df=pd.DataFrame({'Date_Posted':Date,
#                      'Location':Location,
#                      'Title':Title,
#                      'Type':Employment_Type, 
#                      'Salary':Salary,
#                      'Industry':Industry,
#                      'Skills':skills})
#             # Write save it to a csv file
#             #df.to_csv('./page.csv', index = False)

    return df
    
    driver.quit()

In [13]:
page('https://www.seek.com.au/Data-Scientist-jobs?page=1')


20
extracting 0 job keywords...
['python', 'r', 'spark', 'hadoop', 'scala', 'sql'] 4
extracting 1 job keywords...
['python', 'r'] 4


NameError: name 'Location' is not defined

In [314]:
page('https://www.seek.com.au/Data-Scientist-jobs?page=2')
#Searching 'Data Scientist'
#page1 = page(job_page[0])
#page('https://www.seek.com.au/Data-Scientist-jobs?page=2')
#page3 = page(job_page[2])
#page4 = page(job_page[3])
#page5 = page(job_page[4])
#page6 = page(job_page[5])
#page7 = page(job_page[6])

extracting 1 job keywords...
['matlab', 'hadoop', 'python', 'mining', 'r', 'spark', 'sql'] 4


,Date_Posted,Industry,Location,Salary,Skills,Title,Type
0,3 May 2017,Information & Communication Technology,Location: ACT,None,"[matlab, hadoop, python, mining, r, spark, sql]",Data Scientist,Full Time


In [43]:
#Do not run!
#This will run for each page.

for k in range(1,3):
#for k in range(1,num_pages+1):    
#this 5 pages reopen the browser is to prevent connection refused error.
    if k%5==0:
        #driver.quit()
        driver = webdriver.Chrome(executable_path="/Users/gopetro/documents/dsi/code/chromedriver")
        #driver=webdriver.Firefox()
        driver.set_page_load_timeout(15)
    current_url = start_url + str(k)
    print 'extracting %d page of job searching results...' % k
    resp = requests.get(current_url)
    current_soup = BeautifulSoup(resp.content, 'html.parser')
    #current_soup = BeautifulSoup(resp.content)
    #current_urls = current_soup.findAll('a',{'rel':'nofollow','target':'_blank'})
    #current_urls = [link['href'] for link in current_urls]
    
    current_urls =[]
    for entry in start_soup.find_all('a',{'class':'_1OFaluu'},href=True):
        current_urls.append(domain+entry['href'])
    
    for i in range(1,4):
    #for i in range(5,len(urls))
    #for i in range(len(urls)):
        get_info = True
        try:
            #driver.get(base_url+urls[i])
            #driver.get(current_urls[i])
            driver.get(urls[i])
        except TimeoutException:
            get_info = False
        continue
        j = random.randint(1000,2200)/1000.0
        time.sleep(j) #waits for a random time so that the website don't consider you as a bot
        if get_info:
            soup=BeautifulSoup(driver.page_source, 'html.parser')
            print 'extracting %d job keywords...' % i
            single_job = keywords_f(soup)
            print single_job,len(soup)
            print driver.current_url
            job_keywords.append([driver.current_url,single_job])
            skills.append(single_job)
            #Byron's addition
            print 'Date posted'
            print soup.find('strong', {'itemprop': 'datePosted'}).get_text(" ", strip=True)
            Date.append(soup.find('strong', {'itemprop': 'datePosted'}).get_text(" ", strip=True))
            print 'Job title:', soup.find('h1').get_text(" ", strip=True)
            Title.append(soup.find('h1').get_text(" ", strip=True))
            #print(job_page.find('h1', {'class': 'jobtitle'}).get_text(" ", strip=True))
            
            print soup.find('li', {'itemprop': 'jobLocation'}).get_text(" ", strip=True)
            Location.append(soup.find('li', {'itemprop': 'jobLocation'}).get_text(" ", strip=True))
            if soup.find('div', {'itemprop': 'baseSalary'}) == None:
                print 'Salary:', 'None stated'
                Salary.append(soup.find('div', {'itemprop': 'baseSalary'}))
            else:
                print 'Salary:', soup.find('div', {'itemprop': 'baseSalary'}).get_text(" ", strip=True)
                Salary.append(soup.find('div', {'itemprop': 'baseSalary'}).get_text(" ", strip=True))
            
            print 'Employment Type:', soup.find('div', {'itemprop': 'employmentType'}).get_text(" ", strip=True)
            Employment_Type.append(soup.find('div', {'itemprop': 'employmentType'}).get_text(" ", strip=True))
            
            print 'Industry:', soup.find('span', {'itemprop': 'industry'}).get_text(" ", strip=True)
            Industry.append(soup.find('span', {'itemprop': 'industry'}).get_text(" ", strip=True))
# use driver.quit() not driver.close() can get rid of the openning too many files error.
#driver.quit()

extracting 1 page of job searching results...
extracting 2 page of job searching results...


In [316]:
driver.quit()

In [1]:
#skills_dict_1 = [w[1] for w in job_keywords]
#skills_dict_1

skills_dict_2 = [w[1] for w in job_keywords]
skills_dict_2

NameError: name 'job_keywords' is not defined

In [164]:
#Combine into dictionarysdict={}
for words in skills_dict:
    for word in words:
        if not word in sdict:
            sdict[word]=1
        else:
            sdict[word]+=1
Result = pd.DataFrame()
Result['Skill'] = sdict.keys()
Result['Count'] = sdict.values()
Result['Ranking'] = Result['Count']/float(len(job_keywords))

In [165]:
#Convert to DataFrame
Result = pd.DataFrame()
Result['Skill'] = sdict.keys()
Result['Count'] = sdict.values()
Result['Ranking'] = Result['Count']/float(len(job_keywords))

In [154]:
Result

,Skill,Count,Ranking
0,java,33,5.500000
1,rdbms,3,0.500000
2,scala,4,0.666667
3,python,56,9.333333
4,saas,12,2.000000
5,mysql,2,0.333333
6,excel,15,2.500000
7,hadoop,7,1.166667
8,c++,15,2.500000
9,tableau,2,0.333333
